<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install PyPDF2
!pip install dask

In [10]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2
import re
import datetime
import dask

In [3]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [11]:
def list_endpoints():
    date_today = datetime.datetime.now()
    year_today = date_today.year
    month_today = date_today.month
    day_today = date_today.day
    key = "qu2TgXAQPvswtEZ176PFWyfvdb5wd1nwcv2JwEUo"

    endpoints = []
    for i in range(1, 3650):
        past_date = date_today - datetime.timedelta(days=i)
        year = past_date.strftime("%Y")
        month = past_date.strftime("%m")
        day = past_date.strftime("%d")
        endpoint = f"https://api.congress.gov/v3/congressional-record/?y={year}&m={month}&d={day}&api_key={key}"
        endpoints.append(endpoint)
    return endpoints


endpoints_result = list_endpoints()

In [20]:
def find_pdf_links(endpoints_list):
    response_objects = []
    for i in endpoints_list:
        response = dask.delayed(requests.get(i))
        response_objects.append(response)
    return response_objects


delayed_pdf_links = dask.delayed(find_pdf_links)(endpoints_result)
actual_pdf_links = dask.compute(delayed_pdf_links)
display(actual_pdf_links)

([Delayed('Response-42d1619f-8e74-42c7-8a8a-1a8e81d13bea'),
  Delayed('Response-ecc3fb53-b8cf-4197-af7a-66489f5abb99'),
  Delayed('Response-40823115-0b78-4b02-aa4d-6962d9f35b79'),
  Delayed('Response-8e2e2cd9-289b-4f81-b8fb-2e7a0ae81ac6'),
  Delayed('Response-c4d7f9d8-8463-42d9-96bf-f5d78b719e54'),
  Delayed('Response-3d0c9736-eb34-406d-badb-d612962144d9'),
  Delayed('Response-05351aa3-9275-4f5b-a181-3982acd393c8'),
  Delayed('Response-158f1afd-98a1-48fe-b608-0217a3d7b0b6'),
  Delayed('Response-bb768a81-e4c2-4ba1-b7ca-0fcb38673f91'),
  Delayed('Response-37f60656-2b0a-49a3-9665-bafaac93ea1b'),
  Delayed('Response-39baa732-94d8-48db-a269-b5c646a47309'),
  Delayed('Response-687853e2-e891-4ae9-a308-4dc261fe9cc8'),
  Delayed('Response-b2a76c96-887a-400d-8dfe-213fbb5dc675'),
  Delayed('Response-e58c143c-6452-4814-9beb-0096dc97b183'),
  Delayed('Response-4112b161-a41e-4c20-957c-b032e67beb9b'),
  Delayed('Response-f038c88a-a86e-4f46-8343-9a4424875002'),
  Delayed('Response-f94df1d3-0f8d-4443-9

In [23]:
def read_pdf(response_object_list):
  total_matches = []
  for i in response_object_list[0]:
    if i.json()['Results']['Issues'] != []:
      link_to_pdf_file = i.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
      pdf_file = requests.get(link_to_pdf_file)
      pdf_data = BytesIO(pdf_file.content)
      pdf_reader = PyPDF2.PdfReader(pdf_data)
      number_of_pages = len(pdf_reader.pages)
      for j in range(number_of_pages):
        page = pdf_reader.pages[j]
        text = page.extract_text()
        adam_smith_matches = re.findall(r'Adam\w*\W*\sSmith\w*\W*', text)
        if adam_smith_matches != []:
          print(adam_smith_matches)
          print(link_to_pdf_file)
  total_matches += adam_smith_matches
  return total_matches

matches = read_pdf(actual_pdf_links)


TypeError: Truth of Delayed objects is not supported